In [ ]:
import cv2
import collections
import numpy as np
import matplotlib.pyplot as plt
from iplib import *

In [ ]:
# Threshold
def getEdge(img: np.ndarray, threshold) -> np.ndarray:
    edge = cv2.Canny(img, threshold, 255)
    edge[edge < threshold] = 0
    edge[edge >= threshold] = 255

    return edge


def preMask(img):
    b, g, r = cv2.split(img)
    b = cv2.GaussianBlur(b, (5, 5), 0)
    b = binary(b, 0)
    b = morOpen(b, np.ones((11, 11)), 1)
    b = morClose(b, np.ones((21, 21)), 1)
    b = morOpen(b, np.ones((31, 31)), 1)

    masked = extCh(r & b)
    mean = np.sum(masked) / np.sum(masked > 0)
    masked[masked == 0] = mean
    gray = bgr2gray(masked)
    edge = getEdge(gray, 170)

    mask = morClose(edge, np.ones((7, 7)))
    mask = morOpen(mask, np.ones((5, 5)))
    mask = morClose(mask, np.ones((31, 31)))
    mask = morOpen(mask, np.ones((5, 31)))
    mask = morOpen(mask, np.ones((31, 5)))

    return mask


def select(img: np.ndarray, mask: np.ndarray) -> np.ndarray:
    img_crops = []
    boxs = getBox(mask)
    for box in boxs:
        img_crops.append(cropBox(img, box))

    cors = []
    for c in img_crops:
        b, g, r = cv2.split(c)
        h = hist(r)
        cors.append(np.corrcoef(hists[1], h)[0][1])
    sel = np.argmax(cors)

    return img_crops[sel]


def cropVer(img: np.ndarray, threshold: int = 0) -> np.ndarray:
    h, w = img.shape[:2]
    threshold = 0 if threshold == 0 else w // threshold
    vsp = []
    new = True
    for i in range(h):
        s = img[i, :]
        cnt = np.sum(s > 0)
        if(cnt > threshold):
            if new:
                vsp.append(s)
                new = False
            else:
                vsp[-1] = np.vstack([vsp[-1], s])
        else:
            new = True
    sizes = []
    for v in vsp:
        sizes.append(np.prod(v.shape))
    return vsp[np.argmax(sizes)]


def cropHor(img: np.ndarray, threshold: int = 0) -> list[np.ndarray]:
    h, w = img.shape[:2]
    threshold = 0 if threshold == 0 else w // threshold
    kernel = np.array([
        [0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0],
        [1, 1, 1, 1, 1],
        [0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0]
    ], dtype=np.uint8)
    erd = cv2.erode(img, kernel)

    hsp = []
    new = True
    for i in range(w):
        s = np.reshape(erd[:, i], (h, 1))
        cnt = np.sum(s > 0)
        if(cnt > threshold):
            if new:
                hsp.append(s)
                new = False
            else:
                hsp[-1] = np.hstack([hsp[-1], s])
        else:
            new = True

    for i in range(len(hsp)):
        hsp[i] = cv2.dilate(hsp[i], kernel)
        hsp[i] = cv2.resize(hsp[i], (20, 50))
    return hsp


def cropNum(img: np.ndarray):
    vsp = cropVer(img, 4)
    hsp = cropHor(vsp)
    nums = []
    for h in hsp:
        if np.sum(h > 0) > np.prod(h.shape) / 3:
            bg = np.zeros((70, 30), dtype=np.uint8)
            bg[10:-10, 5:-5] = h
            pest = perspectiveTransform(extCh(bg))
            if np.sum(pest > 0) < np.prod(pest.shape) * 0.95:
                nums.append(h)
            # nums.append(bg)
    return nums

def match(img: np.ndarray) -> str:
  img = img.copy()
  img[img > 0] = 255
  scores = [0] * len(templates)
  for i, t in enumerate(templates):
    scores[i] = np.corrcoef(img.ravel(), t.template.ravel())[0][1]
  return templates[np.argmax(scores)].alnum

In [ ]:
imgs = []
for i in range(5):
    imgs.append(imgRead(f"{i}.jpg"))

eqs = []
bs = []
gs = []
rs = []
for i in range(5):
    crop_img = imgRead(f"crop{i}.jpg")
    gray = bgr2gray(crop_img)
    b, g, r = cv2.split(crop_img)
    eqs.append(hist(gray))
    bs.append(hist(b))
    gs.append(hist(g))
    rs.append(hist(r))

bgr_hist = np.sum(eqs, axis=0)
b_hist = np.sum(bs, axis=0)
g_hist = np.sum(gs, axis=0)
r_hist = np.sum(rs, axis=0)

hists = [bgr_hist, r_hist, g_hist, b_hist]

num_img = bgr2gray(imgRead("nums.png"))
alpha_img = bgr2gray(imgRead("alpha.png"))
Alnum = collections.namedtuple("Alnum",['alnum', 'template'])
num_lst = list('1234567890')
alpha_lst = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

# nums = cropNum(num_img)
nums_template = cropVer(num_img, 0)
nums_template = cropHor(nums_template, 0)

alpha_template = cropVer(alpha_img, 0)
alpha_template = cropHor(alpha_template, 0)

kernel = np.array([
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0]
], dtype=np.uint8)
templates = []

for n, t in zip(num_lst, nums_template):
  t[t > 0] = 255
  templates.append(Alnum(n, t))
for n, t in zip(alpha_lst, alpha_template):
  t[t > 0] = 255
  templates.append(Alnum(n, t))

ts = []
for n, t in templates:
  ts.append(t)
imgPlotTable(ts, 3, 12)

In [ ]:
def test():
  for img in imgs:
    mask = preMask(img)
    sel = select(img, mask)
    pes = perspectiveTransform(sel)
    bina = binary(bgr2gray(cv2.GaussianBlur(pes, (3,3), 0)), 0)
    lst = [img, img & extCh(mask), sel, pes, bina]
    nums = cropNum(bina)
    figSize(10,20)
    imgPlotTable(lst, 1, len(lst))
    plt.show()
    figSize(5,5)
    imgPlotTable(nums, 1, len(nums))
    plt.show()
    for num in nums:
      print(match(num), end = ' ')
    print()

test()